# Processamento de Linguagem Natural - Trabalho Prático 1
### Thaís Ferreira da Silva - 2021092571

### Import

In [2]:
# Imports do gensim - para word2vec
import gensim
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus

# Imports essenciais
from scipy.spatial.distance import cosine
import os
import random

### Funções Auxiliares

In [3]:
def generate_hyperparameter_combinations(param_grid):
    from itertools import product
    keys, values = zip(*param_grid.items())
    combinations = [dict(zip(keys, v)) for v in product(*values)]
    return combinations


In [4]:
def train_and_save_model(corpus, params, output_dir):
    model_name = f"word2vec_vs{params['vector_size']}_win{params['window']}_sg{params['sg']}_ep{params['epochs']}"
    print(f"Treinando modelo: {model_name}")
    
    model = Word2Vec(
        sentences=corpus,
        vector_size=params['vector_size'],
        window=params['window'],
        sg=params['sg'],
        epochs=params['epochs']
    )
    
    model_path = os.path.join(output_dir, f"{model_name}.model")
    model.save(model_path)
    print(f"Modelo salvo: {model_path}")
    
    return model

In [5]:
def get_analogies(model, questions_words_path):
    final_analogies = []
    with open(questions_words_path, 'r') as f:
        for line in f:
            if line.startswith(':'):
                continue

            words = [word.lower().strip() for word in line.split()]

            final_words = [word for word in words if word in model.wv]
            #final_words = [word if word in model.wv else None for word in words]
            if len(final_words) != 4:
                continue
            
            final_analogies.append(final_words)
    
    random.shuffle(final_analogies)
    return final_analogies

In [6]:
def evaluate_models(model, analogies):
    avg_distance = 0
    correct = 0
    count = 0
    accuracy = 0

    #print("numero de analogias ", len(analogies))

    for analogy in analogies:
        if len(analogy) == 4:
            count += 1
            result_vector = model.wv[analogy[1]] - model.wv[analogy[0]] + model.wv[analogy[2]]
            predicted = model.wv.similar_by_vector(result_vector, topn=20, restrict_vocab=None)
            predicted_word = next((word for word, _ in predicted if word not in analogy[:3]), None)
            if analogy[3] == predicted_word:
                correct += 1
            # else:
            #     print(f"Analogias: {analogy}, Previsão: {predicted}, Palavra prevista: {predicted_word}, Esperado: {analogy[3]}\n")
            cosine_distance = cosine(model.wv[analogy[3]], model.wv[predicted_word])
            avg_distance += cosine_distance
            
    accuracy = correct/len(analogies)
    avg_distance /= count
    return accuracy, avg_distance

### Preprocessamento do dados de treino

In [10]:
text8_path = './text8'
corpus = Text8Corpus(text8_path)

sentence = next(iter(corpus))
print(sentence[:50])

output_dir = './word2vec_models'
os.makedirs(output_dir, exist_ok=True)

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the']


### Treinamento do modelo Word2Vec

In [ ]:
# Hiperparâmetros para o GridSearch
# param_grid = {
#     'vector_size': [50, 100, 200],      # Tamanho do vetor de palavras
#     'window': [3, 5, 7],                # Tamanho da janela de contexto
#     'sg': [0, 1],                      # CBOW (0) ou Skip-gram (1)
#     'epochs': [5, 10, 15],             # Número de iterações de treinamento
# }

param_grid = {
    'vector_size': [50],      # Tamanho do vetor de palavras
    'window': [3, 5, 7],                # Tamanho da janela de contexto
    'sg': [0, 1],                      # CBOW (0) ou Skip-gram (1)
    'epochs': [5],             # Número de iterações de treinamento
}

combinations = generate_hyperparameter_combinations(param_grid)

for i, params in enumerate(combinations):
    print(f"\nTreinando combinação {i+1}/{len(combinations)}: {params}\n")
    model = train_and_save_model(corpus, params, output_dir)


Treinando combinação 1/6: {'vector_size': 50, 'window': 3, 'sg': 0, 'epochs': 5}

Treinando modelo: word2vec_vs50_win3_sg0_ep5
Modelo salvo: ./word2vec_models/word2vec_vs50_win3_sg0_ep5.model

Treinando combinação 2/6: {'vector_size': 50, 'window': 3, 'sg': 1, 'epochs': 5}

Treinando modelo: word2vec_vs50_win3_sg1_ep5
Modelo salvo: ./word2vec_models/word2vec_vs50_win3_sg1_ep5.model

Treinando combinação 3/6: {'vector_size': 50, 'window': 5, 'sg': 0, 'epochs': 5}

Treinando modelo: word2vec_vs50_win5_sg0_ep5
Modelo salvo: ./word2vec_models/word2vec_vs50_win5_sg0_ep5.model

Treinando combinação 4/6: {'vector_size': 50, 'window': 5, 'sg': 1, 'epochs': 5}

Treinando modelo: word2vec_vs50_win5_sg1_ep5
Modelo salvo: ./word2vec_models/word2vec_vs50_win5_sg1_ep5.model

Treinando combinação 5/6: {'vector_size': 50, 'window': 7, 'sg': 0, 'epochs': 5}

Treinando modelo: word2vec_vs50_win7_sg0_ep5
Modelo salvo: ./word2vec_models/word2vec_vs50_win7_sg0_ep5.model

Treinando combinação 6/6: {'vector

### Avaliação do modelo treinado

In [11]:
model = Word2Vec.load('./word2vec_models/word2vec_vs50_win2_sg1_ep2.model')
questions_words_path = './mini-questions-words.txt'

analogies = get_analogies(model, questions_words_path)

accuracy, avg_distance = evaluate_models(model, analogies)
print(f"vector_size: {model.vector_size}, window: {model.window}, sg: {model.sg}, epochs: {model.epochs} ---> Accuracy: {accuracy}, avg_distance: {avg_distance}")


vector_size: 50, window: 2, sg: 1, epochs: 2 ---> Accuracy: 0.13636363636363635, avg_distance: 0.11626363779315324


In [12]:
#para cada modelo treinado, avaliar as analogias
#pegar os modelos da pasta
questions_words_path = './questions-words.txt'

models = os.listdir(output_dir)
models = [model for model in models if model.endswith('.model')]

for model_name in models:
    analogies = get_analogies(model, questions_words_path)
    model = Word2Vec.load(os.path.join(output_dir, model_name))
    accuracy, avg_distance = evaluate_models(model, analogies)
    #avg_distance -> quanto menor melhor
    print(f"vector_size: {model.vector_size}, window: {model.window}, sg: {model.sg}, epochs: {model.epochs} ---> Accuracy: {accuracy}, avg_distance: {avg_distance}")


vector_size: 50, window: 2, sg: 1, epochs: 2 ---> Accuracy: 0.1141526897402816, avg_distance: 0.20041507979685003
vector_size: 50, window: 7, sg: 0, epochs: 5 ---> Accuracy: 0.13917092051382735, avg_distance: 0.3845468280240983
vector_size: 50, window: 7, sg: 1, epochs: 5 ---> Accuracy: 0.23122230324788243, avg_distance: 0.23943688070535885
vector_size: 50, window: 2, sg: 1, epochs: 1 ---> Accuracy: 0.06394794412969092, avg_distance: 0.16693464503625816
vector_size: 50, window: 5, sg: 0, epochs: 5 ---> Accuracy: 0.14057328770965388, avg_distance: 0.3725509322484953
vector_size: 50, window: 3, sg: 0, epochs: 5 ---> Accuracy: 0.13743198519100241, avg_distance: 0.3562493044272802
vector_size: 50, window: 3, sg: 1, epochs: 5 ---> Accuracy: 0.19515341897122343, avg_distance: 0.23734800473890358
vector_size: 50, window: 5, sg: 1, epochs: 5 ---> Accuracy: 0.2184888091097773, avg_distance: 0.23817156122137115
